In [15]:
import yfinance as yf
import polars as pl

In [16]:
# Define tickers for American and German auto brands and stock indices
tickers = ["TSLA", "F", "GM", "VWAGY", "BMWYY", "MBGYY", "^GSPC", "^GDAXI"]

# Map each stock to its respective stock market index
index_mapping = {
    "TSLA" : "^GSPC",  # American stocks -> S&P 500
    "F"    : "^GSPC",
    "GM"   : "^GSPC",
    "VWAGY": "^GDAXI",  # German stocks -> DAX
    "BMWYY": "^GDAXI",
    "MBGYY": "^GDAXI",
}

In [17]:
data = yf.download(tickers, start="2005-01-01", end = "2024-10-31", group_by="ticker")

[                       0%                       ]

[                       0%                       ]

[******************    38%                       ]  3 of 8 completed

[**********************50%                       ]  4 of 8 completed

[**********************62%*****                  ]  5 of 8 completed

[*********************100%***********************]  8 of 8 completed

In [18]:
# Extract the necessary data in long format
long_format = []

for ticker in tickers:
    if ticker.startswith("^"):                    # Skip index tickers for now
        continue
    ticker_data = data[ticker].reset_index()      # Extract data and reset index
    ticker_data = ticker_data[["Date", "Close"]]  # Keep only date and close price
    ticker_data["Symbol"] = ticker                # Add stock symbol as a column

    # Add the stock market index (based on mapping)
    ticker_data["Index"] = index_mapping[ticker]

    # Append to the long format list
    long_format.append(pl.DataFrame(ticker_data))

# Combine all stock data into one Polars DataFrame
stocks_long = pl.concat(long_format)

In [19]:
# Download indices data separately
indices_data = yf.download(["^GSPC", "^GDAXI"], start = "2005-01-01", end="2024-10-31", group_by="ticker")
indices_long = []

for index in ["^GSPC", "^GDAXI"]:
    index_data = indices_data[index].reset_index()  # Extract index data and reset index
    index_data = index_data[["Date", "Close"]]  # Keep only date and close price
    index_data["Index"] = index  # Add index name as a column
    indices_long.append(pl.DataFrame(index_data))

# Combine all index data into one Polars DataFrame
indices_long = pl.concat(indices_long).rename({"Close": "Index_Close"})
indices_long

[                       0%                       ]

[*********************100%***********************]  2 of 2 completed

Date,Index_Close,Index
"datetime[ns, UTC]",f64,str
2005-01-03 00:00:00 UTC,1202.079956,"""^GSPC"""
2005-01-04 00:00:00 UTC,1188.050049,"""^GSPC"""
2005-01-05 00:00:00 UTC,1183.73999,"""^GSPC"""
2005-01-06 00:00:00 UTC,1187.890015,"""^GSPC"""
2005-01-07 00:00:00 UTC,1186.189941,"""^GSPC"""
…,…,…
2024-10-24 00:00:00 UTC,19443.0,"""^GDAXI"""
2024-10-25 00:00:00 UTC,19463.589844,"""^GDAXI"""
2024-10-28 00:00:00 UTC,19531.619141,"""^GDAXI"""


In [20]:
car_names = pl.DataFrame({
    "Symbol": ["TSLA", "F", "GM", "VWAGY", "BMWYY", "MBGYY"],
    "Company": ["Tesla", "Ford", "General Motors", "Volkswagen", "BMW", "Mercedes-Benz"]
})
car_names

Symbol,Company
str,str
"""TSLA""","""Tesla"""
"""F""","""Ford"""
"""GM""","""General Motors"""
"""VWAGY""","""Volkswagen"""
"""BMWYY""","""BMW"""
"""MBGYY""","""Mercedes-Benz"""


In [21]:
# Join stock data with the respective index data
stocks_with_index = (
    stocks_long
    .join(indices_long, on = ["Date", "Index"], how = "inner")
    .join(car_names,    on = ["Symbol"],        how = "inner")
    .with_columns(
        stock_return  = pl.col('Close').pct_change().over('Symbol'),
        market_return = pl.col('Index_Close').pct_change().over('Symbol')
    )
    
)
stocks_with_index

Date,Close,Symbol,Index,Index_Close,Company,stock_return,market_return
"datetime[ns, UTC]",f64,str,str,f64,str,f64,f64
2005-01-03 00:00:00 UTC,null,"""TSLA""","""^GSPC""",1202.079956,"""Tesla""",null,null
2005-01-04 00:00:00 UTC,null,"""TSLA""","""^GSPC""",1188.050049,"""Tesla""",null,-0.011671
2005-01-05 00:00:00 UTC,null,"""TSLA""","""^GSPC""",1183.73999,"""Tesla""",null,-0.003628
2005-01-06 00:00:00 UTC,null,"""TSLA""","""^GSPC""",1187.890015,"""Tesla""",null,0.003506
2005-01-07 00:00:00 UTC,null,"""TSLA""","""^GSPC""",1186.189941,"""Tesla""",null,-0.001431
…,…,…,…,…,…,…,…
2024-10-24 00:00:00 UTC,15.73,"""MBGYY""","""^GDAXI""",19443.0,"""Mercedes-Benz""",0.020766,0.003374
2024-10-25 00:00:00 UTC,15.48,"""MBGYY""","""^GDAXI""",19463.589844,"""Mercedes-Benz""",-0.015893,0.001059
2024-10-28 00:00:00 UTC,15.48,"""MBGYY""","""^GDAXI""",19531.619141,"""Mercedes-Benz""",0.0,0.003495


In [22]:
stocks_with_index.write_csv('stocks_with_index.csv')